# Normality index

## Connect to the database

In [ ]:
from databaseconnection import DatabaseConnection
gds = DatabaseConnection().get_database_connection()
gds.version()

## Create a normality index.

Create a normality index for each respondent in the data set. The normality index is calculated with the following formula:

$$
\frac{1}{n}\sum_{i=1}^{n}{p_i}
$$

Where $n$ is the number of questions answered by the respondent and $p_i$ is the percentage of respondents that have chosen the alternative for question $i$.

In [ ]:
gds.run_cypher("""
                MATCH (re:Respondent)-[:HAS_ANSWERED]->(a:Answer)-[:CHOSE]->(qa:QuestionAlternative)<-[:CONSISTS_OF]-(q:Question)
                WITH re, q, qa, COUNT(a) as answersForAlternative
                MATCH (q)-[:CONSISTS_OF]->(qa_total:QuestionAlternative)
                WITH re, q, qa, answersForAlternative, COUNT(qa_total) as totalAlternatives
                WITH re, q, toFloat(answersForAlternative) / toFloat(totalAlternatives) as p_i
                WITH re, COUNT(q) as totalQuestions, SUM(p_i) as sum_p_i
                SET re.normalityIndex = sum_p_i / totalQuestions
                RETURN re.id as respondentId, re.normalityIndex as normalityIndex
               """)